Just a debugging & samples notebook

In [17]:


# read structure and sequence from ct file 
from  sincfold_utils.io import read_ct, ct2dot, bp2dot
seq, bp = read_ct('samples/sample.ct')
print(seq)

from sincfold_utils.validate import is_valid_sequence
print("is seq valid: ", is_valid_sequence(seq))

print(bp)
print(ct2dot('samples/sample.ct'))

bp_missing = bp.copy()
bp_missing.pop(0)
print(bp2dot(seq, bp_missing))
bp_extra = bp.copy()
bp_extra.append([9, 15])
print(bp2dot(seq, bp_extra))

bp_1miss = bp.copy()
bp_1miss.pop(10)
bp_1miss.append([14,22])
print(bp2dot(seq, bp_1miss))

GGGGGCGUAGCUCAGAUGGUAGAGCGCUCGCUUGGCGUGUGAGAGGUACCGGGAUCGAUACCCGGCGCCUCCACCA
is seq valid:  True
[[1, 72], [2, 71], [3, 70], [4, 69], [5, 68], [6, 67], [7, 66], [10, 25], [11, 24], [12, 23], [13, 22], [27, 43], [28, 42], [29, 41], [30, 40], [31, 39], [49, 65], [50, 64], [51, 63], [52, 62], [53, 61]]
(((((((..((((........)))).(((((.......))))).....(((((.......))))))))))))....
.((((((..((((........)))).(((((.......))))).....(((((.......))))))))))).....
(((((((.<((((.>......)))).(((((.......))))).....(((((.......))))))))))))....
(((((((..(((.(.......)))).(((((.......))))).....(((((.......))))))))))))....


In [18]:
from sincfold_utils.metrics import bp_metrics

print("f1, recall, precision")
print(bp_metrics(bp, bp))
print(bp_metrics(bp, bp_missing))
print(bp_metrics(bp, bp_missing, strict=False))

print(bp_metrics(bp, bp_1miss))
print(bp_metrics(bp, bp_1miss, strict=False))


f1, recall, precision
(1.0, 1.0, 1.0)
(0.975609756097561, 0.9523809523809523, 1.0)
(0.975609756097561, 0.9523809523809523, 1.0)
(0.9523809523809523, 0.9523809523809523, 0.9523809523809523)
(1.0, 1.0, 1.0)


In [2]:
from sincfold_utils.predictors import fold
!rm 20*
lppath = "../LinearPartition/linearpartition" # TODO this should be a config arg
with open("samples/trna.fasta") as fin:
    id = fin.readline().strip()
    seq = fin.readline().strip()
    ref = fin.readline().strip()

print(ref)
print("------")    
print(fold(seq, "linearpartition", convert_to_bp=False))
print(fold(seq, "RNAstructure", convert_to_bp=False))


(((((((..((((........))))((((((.......))))))...(((((.......))))))))))))....
------
('(((((((((((((........)))..(((((.......)))))(((((((...))).))))))))))))))....', -31.35745)
Initializing nucleic acids...done.
Setting temperature...done.
 92% [===============================================   ] |                     done.
Writing output ct file...done.
Single strand folding complete.
('(((((((((((((........))).....(((((((......)))))))(((......)))))))))))))....', -38.5)


In [1]:
from sincfold_utils.metrics import WL
import numpy as np 
y = np.zeros((10, 10))
for p in [[0, 9], [1, 8], [2, 7], [3, 6], [4, 5]]:
    y[p[0], p[1]] = 1
    y[p[1], p[0]] = 1
yhat = y.copy()
yhat[0, 9]=0
yhat[9, 0]=0

yhat2 = np.zeros((10, 10))
for p in [[1, 9], [2, 8], [3, 7], [4, 6]]:
    yhat2[p[0], p[1]] = 1
    yhat2[p[1], p[0]] = 1

# pseudoknot ({(((}))))
yps = np.zeros((10, 10))
for p in [[0, 9], [1, 5], [4, 6], [3, 7], [2, 8]]:
    yps[p[0], p[1]] = 1
    yps[p[1], p[0]] = 1

WL(yhat, y), WL(y, y), WL(yhat2, y),  WL(yps, y)

(np.float64(0.8333333333333334),
 np.float64(1.0),
 np.float64(0.16666666666666666),
 np.float64(0.3333333333333333))

In [2]:
from sincfold_utils.io import mat2dot
y = mat2dot(y)
yhat = mat2dot(yhat)
yhat2 = mat2dot(yhat2)
yps = mat2dot(yps)
y, yhat, yhat2, yps

('((((()))))', '.(((()))).', '.((((.))))', '(<(((>))))')

In [3]:
from sincfold_utils.metrics import rnadistance

rnadistance(y, y), rnadistance(y, yhat), rnadistance(y, yhat2), rnadistance(y, yps) 

(0.0, 0.4, 0.2, 0.2)

In [14]:
ls ../RNAstructure/RNAdistance

ls: no se puede acceder a '../RNAstructure/RNAdistance': No existe el fichero o el directorio


In [ ]:
# tests to run
#- f1 equivalencies between functions, and with known values
# read/write ct files